## **0. Download dataset**
**Note:** If you can't download using gdown due to limited number of downloads, please download it manually and upload it to your drive, then copy it from the drive to colab.
```python
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .
```

In [ ]:
# https://drive.google.com/file/d/1aQ8OlUljwEm7RLZz8Qf4xzim0nDewHfU/view?usp=sharing
!gdown --id 1aQ8OlUljwEm7RLZz8Qf4xzim0nDewHfU

In [ ]:
!unzip ../content/twitter_sentiment_analysis_3cls_dataset.zip

## **1. Import libraries**

## **2. Read dataset**

## **3. Drop missing value**

## **4. Preprocessing data**



## **5. One-hot encoding label**

## **6. Create train, val, test set**

## **7. Define Softmax Regression model**

## **8. Training**

## **9. Evaluation**